In [1]:
import numpy as np
import scipy.sparse as sp
import time
import os
import psutil
import shelve
from pybdm import BDM
from pybdm.utils import decompose_dataset
from joblib import Parallel, delayed
from node import NodePerturbationExperiment
from algorithms import PerturbationExperiment

from itertools import product

In [ ]:
%pylab inline

In [7]:
N = 1000
b = np.random.randint(0,2,(N,N))
X = ((b + b.T)/2).astype(int)
bdm = BDM(ndim=2)

In [8]:
perturbation = PerturbationExperiment(bdm, X, bipartite_network=False)
start = time.time()
perturbation.node_equivalent()
print('Time:',time.time()-start)

Time: 61.71534276008606


In [ ]:
edge_exp = PerturbationExperiment(bdm,X)
start = time.time()
value = edge_exp.run()
print('Time:',time.time()-start)
print(value[:5])

In [ ]:
edge_exp = PerturbationExperiment(bdm,X)
start = time.time()
indexes = [ range(k) for k in X.shape ]
idx = np.array([ x for x in product(*indexes) ], dtype=int)
l = X.size
values = np.full((l, ), -1, dtype=int)
results = Parallel(n_jobs=4)(delayed(edge_exp._method)(tuple(idx[i,:]),values[i])
                             for i in range(l))
print('Time:',time.time()-start)
print(results[:5])

In [ ]:
figure(figsize=(8,12))
plot(value,linewidth=10, alpha=0.7,label='series')
plot(results,label='parallel')
legend()
show()

In [ ]:
plot(value)
print(value[-10:])

In [ ]:
mat_v = value.reshape(X.shape)

In [ ]:
print(sum(mat_v[:,2]),mean(mat_v[:,2]))
print(sum(mat_v[2,:]),mean(mat_v[2,:]))

In [ ]:
avg = np.array([[sum(mat_v[:,i]),mean(mat_v[:,i]),sum(mat_v[i,:]),mean(mat_v[i,:]),
                 (mean(mat_v[i,:])+mean(mat_v[:,i]))/2]
        for i in range(X.shape[0])])

In [ ]:
figure(figsize=(8,12))
plot(avg[:,0],linewidth=10, alpha=0.7,label='Row sum')
plot(avg[:,1],linewidth=10, alpha=0.7,label='Row avg')
plot(avg[:,2],label='Col sum')
plot(avg[:,3],label='Col avg')
#plot(avg[:,4],label='avg')
legend()
show()

## Tests parallelization node experiments

In [ ]:
np.random.seed(303)
X = np.random.randint(0,2,(500,300))
bdm = BDM(ndim=2)

In [ ]:
start = time.time()
val = bdm.ent(X)
print(time.time()-start)
print(val)

In [ ]:
start = time.time()
counters = Parallel(n_jobs=8) \
    (delayed(bdm.decompose_and_count)(d) for d in decompose_dataset(X, (500, 500)))
val = bdm.compute_ent(*counters)
print(time.time()-start)
print(val)

In [ ]:
test_series = NodePerturbationExperiment(bdm,X,metric='bdm',bipartite_network=True)

In [ ]:
start = time.time()
nodes = test_series.run()
print(time.time()-start)
print(nodes[0])

In [ ]:
test_parallel = NodePerturbationExperiment(bdm,X,parallel=True, jobs=8,bipartite_network=True)

In [ ]:
start = time.time()
nodes = test_parallel.run()
print(time.time()-start)
print(nodes[0])